<a href="https://colab.research.google.com/github/AmbiTyga/Task-Stylumia/blob/Basic/Model-Training_and_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/AmbiTyga/Task-Stylumia/Basic/Final.7z

--2021-02-27 17:17:06--  https://raw.githubusercontent.com/AmbiTyga/Task-Stylumia/Basic/Final.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7811917 (7.4M) [application/octet-stream]
Saving to: ‘Final.7z’

Final.7z            100%[===================>]   7.45M  --.-KB/s    in 0.1s    

2021-02-27 17:17:07 (78.0 MB/s) - ‘Final.7z’ saved [7811917/7811917]



In [2]:
!7z x Final.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 7811917 bytes (7629 KiB)

Extracting archive: Final.7z
--
Path = Final.7z
Type = 7z
Physical Size = 7811917
Headers Size = 194
Method = LZMA2:24
Solid = +
Blocks = 1

  0%     40% - test_parsed.json                        80% 1 - train_parsed.json                          Everything is Ok

Files: 2
Size:       31295597
Compressed: 7811917


In [62]:
import pandas as pd
import numpy as np
import torch 
import torchtext
import torch.nn as nn
import re

In [75]:
train = pd.read_json('train_parsed.json')

TypeError: ignored

In [76]:
train['title'].fillna('NULL',inplace = True)
train['body'].fillna('NULL',inplace = True)

In [77]:
train['title'] = train['title'].apply(lambda text:re.sub(r'[\W_]+',' ', text))
train['body'] = train['body'].apply(lambda text:re.sub(r'[\W_]+',' ', text))

In [78]:
train['tld_with_tok'] = train['tld'].apply(lambda x:'<tld> '+x)
train['ac_tok'] = train['alchemy_category'].apply(lambda x:'<ac> '+x)
train['title_tok'] = train['title'].apply(lambda x:'<title> '+x)
train['body_tok'] = train['body'].apply(lambda x:'<body> '+x)

In [79]:
train[[x for x in train.columns if 'tok' in x]]

,tld_with_tok,ac_tok,title_tok,body_tok
0,<tld> com,<ac> business,<title> IBM Sees Holographic Calls Air Breathi...,<body> A sign stands outside the International...
1,<tld> com,<ac> recreation,<title> The Fully Electronic Futuristic Starti...,<body> And that can be carried on a plane with...
2,<tld> com,<ac> health,<title> Fruits that Fight the Flu Men s Health,<body> Apples The most popular source of antio...
3,<tld> com,<ac> health,<title> 10 Foolproof Tips for Better Sleep by ...,<body> There was a period in my life when I ha...
4,<tld> com,<ac> sports,<title> The 50 Coolest Jerseys You Didn t Know...,<body> Jersey sales is a curious business Whet...
...,...,...,...,...
7390,<tld> com,<ac> computer_internet,<title> Kno Raises 46 Million More To Build Mo...,<body> Marc Andreessen is normally enthusiasti...
7391,<tld> com,<ac> culture_politics,<title> Why I Miss College,<body> Mar 30 2009 I d like to congratulate Ja...
7392,<tld> com,<ac> recreation,<title> Sweet Potatoes Eat This Not That,<body> They re loaded with vitamin C which smo...
7393,<tld> com,<ac> arts_entertainment,<title> Naturally Ella,<body>


In [81]:
train[train['title'].str.len()==39]['title']

75      The Baseball Collector The glove trick 
163     9 Habits That Drain Our Energy And Time
221     Vertical Layer Cake Tutorial i am baker
503     Chocolate Malt Waffles The Naptime Chef
546     Hovercraft ready to take off News Video
                         ...                   
6974    The Psychology of Happiness Real Simple
7066     love Maegan Fashion DIY Home Lifestyle
7290    Video Breaking News Videos from CNN com
7349    Confessions of a Tart Strawberry Scones
7355    The Adult Grilled Cheese recipegirl com
Name: title, Length: 122, dtype: object

In [80]:
max([len(x.split()) for x in train['title'].values])

39

In [26]:
train['is_news'].replace('?','0',inplace = True)
train['is_news'] = train['is_news'].apply(int)

In [30]:
train['news_front_page'].value_counts()

0    5853
?    1248
1     294
Name: news_front_page, dtype: int64

In [35]:
train.iloc[:4,:]

,org_url,urlid,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,real_url,domain,tld,body,title
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0,0.090774,0,0.245831,0.003883,1,1,24,0,5424,170,8,0.152941,0.079130,0.0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,bloomberg.com,com,A sign stands outside the International Busine...,"IBM Sees Holographic Calls, Air-Breathing Batt..."
1,http://www.popsci.com/technology/article/2012-...,8471,recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0,0.098707,0,0.203490,0.088652,1,1,40,0,4973,187,9,0.181818,0.125448,1.0,http://www.popsci.com/technology/article/2012-...,popsci.com,com,And that can be carried on a plane without the...,"The Fully Electronic, Futuristic Starting Gun ..."
2,http://www.menshealth.com/health/flu-fighting-...,1164,health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0,0.072448,0,0.226402,0.120536,1,1,55,0,2240,258,11,0.166667,0.057613,1.0,http://www.menshealth.com/health/flu-fighting-...,menshealth.com,com,Apples The most popular source of antioxidants...,Fruits that Fight the Flu | Men's Health
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,0.480725,0.0,0,0.095861,0,0.265656,0.035343,1,0,24,0,2737,120,5,0.041667,0.100858,1.0,http://www.dumblittleman.com/2007/12/10-foolpr...,dumblittleman.com,com,There was a period in my life when I had a lot...,10 Foolproof Tips for Better Sleep - by Dumb L...


In [42]:
chunk = pd.concat([x.iloc[:2,:] for _,x in train.groupby('news_front_page')],ignore_index=True)

In [51]:
train.select_dtypes('int').value_counts()

urlid  framebased  hasDomainLink  is_news  lengthyLinkDomain  linkwordscore  non_markup_alphanum_characters  numberOfLinks  numwords_in_url
10566  0           0              1        0                  2              21692                           139            5                  1
3535   0           0              1        1                  38             1522                            94             4                  1
3538   0           0              0        0                  3              661                             6              2                  1
3539   0           0              1        1                  49             4260                            298            1                  1
3543   0           0              0        1                  26             3702                            120            8                  1
                                                                                                                                       

In [41]:
train['label'] = train['label'].apply(bool)